# Utility to publish Power BI reports in a workspace
#### Configuring libraries and defining constants

In [ ]:
import requests, pandas as pd
from azure.identity import ClientSecretCredential
from pyspark.sql.functions import lit

In [ ]:
tenant_id = 'your-1111-aaaa-bbbb-tenantid'

client_id = dbutils.secrets.get(scope = 'key-vault-dbxsecrets', key = 'next-level-bi-sp-appid')
client_secret = dbutils.secrets.get(scope = 'key-vault-dbxsecrets', key = 'next-level-bi-sp-secret')

pbi_scope = 'https://analysis.windows.net/powerbi/api/.default'
BASE_URL = 'https://api.powerbi.com/v1.0/myorg/'

## Authorization

In [ ]:
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                                                        tenant_id = tenant_id,
                                                        client_id = client_id,
                                                        client_secret = client_secret)

access_token = auth.get_token(pbi_scope)
token = access_token.token
AUTHORIZATION = {'Authorization': f'Bearer {token}'}

## Defining and executing function to publish Power BI report

In [ ]:
def publish_powerbi_report(workspace_id: str, report_name: str, report_content):
    payload = {"datasetDisplayName": report_name,
               "nameConflict":"CreateOrOverwrite"
               }
    files = {report_name: report_content}
    return requests.post(url=f"{BASE_URL}groups/{workspace_id}/imports",headers=AUTHORIZATION,params=payload, files=files).json()

In [ ]:
group_id = '1234567-1234-4567-0987-12345678901'
with open('/dbfs/mnt/landing-zone/Next Level BI.pbix', 'rb') as f:
    print(publish_powerbi_report(group_id, "Next Level BI.pbix",f))